# Food Hunt in downtown Toronto (Part 2)

# Load Packages

In [7]:
!pip install folium

     |████████████████████████████████| 102kB 6.9MB/s ta 0:00:011


In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import folium
from folium.plugins import MarkerCluster
from geopy.geocoders import Nominatim

In [10]:
def locator(df, postal_code, country_code):
    import pgeocode
    import pandas

    nomi = pgeocode.Nominatim(country_code)
    df['lat'] = df[postal_code].apply(lambda x:nomi.query_postal_code(x).latitude)
    df['lng'] = df[postal_code].apply(lambda x:nomi.query_postal_code(x).longitude)

    return df

In [12]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url, 'lxml')

In [13]:
print(soup.prettify()[:1000])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"effb7f1c-76f8-4e4a-96e6-cb5115a410a2","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":960187814,"wgRevisionId":960187814,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toron

In [14]:
wikitable = soup.find('table')
wikitable.find_all('th')
for i in wikitable.find_all('th'):
    print(i.text.strip())

Postal Code
Borough
Neighborhood


In [ ]:
columns = [i.text.strip() for i in wikitable.find_all('th')]
data = [[j.text.strip() for j in i.find_all('td')] for i in wikitable.find_all("tr")[1:]]
trtdata = pd.DataFrame(data, columns = columns)
trtdata['Neighborhood'] = trtdata.Neighborhood.apply(lambda x: x.replace(' /',','))
trtdata.columns = ['PostalCode','Borough', 'Neighborhood']


In [2]:
# The code was removed by Watson Studio for sharing.